# **GeoFlood demo Jupyter notebook (local)**

**Purpose of this notebook**
 - Perform all operations in the GeoFlood workflow to obtain flood inundation extent and depth based on a hydrological forecast of discharge.

---
# **(1) Prepare the GeoFlood software environment**
 - Install GRASS GIS
   - Refer to included GRASS_ubuntu_installation.txt if running Ubuntu 20.04
 - Create GeoFlood conda environment and use as kernel for this notebook
   - See included geoflood-env.yml
 - Install TauDEM from GitHub
   - https://github.com/markwang0/TauDEM (my fork, tested with this notebook)
   - https://github.com/dtarb/TauDEM (original)

---
# **(2) Configuration of folders for storing and accessing inputs and outputs**

1. Make a folder called `geoflood_demo` to store input and output data for this demo notebook
2. Extract example input data to `geoflood_demo`

In [ ]:
%%bash

tar xzf INPUT.tar.gz INPUT 2> /dev/null
find INPUT -name '.*' -type f -delete
mkdir geoflood_demo
mv INPUT geoflood_demo

Clone GeoFlood repo in `geoflood_demo` directory

In [ ]:
%%bash

cd geoflood_demo
git clone https://github.com/passaH2O/GeoFlood.git

This configuration script sets up the GeoFlood file structure in our `geoflood_demo` folder

In [ ]:
!python geoflood_demo/GeoFlood/GeoNet/pygeonet_configure.py \
  -dir geoflood_demo \
  -p OnionCreek -n OC1mTest --no_chunk \
  --input_dir INPUT --output_dir OUTPUT


---
# **(3) Run nonlinear filtering on a portion of Onion Creek**

In [ ]:
!python geoflood_demo/GeoFlood/GeoNet/pygeonet_nonlinear_filter.py

---
# **(4) Compute flow directions and flow accumulation**
We are now going to use GRASS GIS functions to compute flow directions, flow accumulation, and identify subbasins within our domain based on this information. All these operations are contained within the script `pygeonet_grass_py3.py`. As usual, you can open it from your cloned repo on the left to have a look. 

In [ ]:
!python geoflood_demo/GeoFlood/GeoNet/pygeonet_grass_py3.py

---
# **(5) Compute slope and geometric curvature on the filtered terrain**

We are now going to call as input our filtered landscape and we will compute the slope and geometric curvature, which is the default definition of curvature used in GeoNet and GeoFlood. All the operations are contained within `pygeonet_slope_curvature.py`. You can open the actual script by double clicking on it from the repository on the left.

In [ ]:
!python geoflood_demo/GeoFlood/GeoNet/pygeonet_slope_curvature.py

---
# **(6) Compute skeletons based on curvature, flow accumulation, and curvature and flow accumulation combined**

We will now call a new operation in this notebook, which allows us to use curvature only, flow accumulation only, and curvature and flow accumulation combined to identify which parts of the landscape are likely channelized. All these operations are contained within `pygeonet_skeleton_definition.py`.

In [ ]:
!python geoflood_demo/GeoFlood/GeoNet/pygeonet_skeleton_definition.py

---
# **(7) Identify end points of NHD MR channels**

We are going to now use `Network_Node_reading.py` to automatically identify the end points of the input flowline - the NHD MR information for this watershed. This operation is contained within the GeoFlood portion of the code and will generate a new output in the Output folder called `OC1mTest_endPoints`. This file contains the coordinates (start and end point) of the end points of each flowline. 

In [ ]:
!python geoflood_demo/GeoFlood/GeoFlood/Network_Node_Reading.py

---
# **(8) Network extraction with cost function including negative height**

We are now going to extract the geodesic curve between end points using the default cost function that includes negative height.  

**Compute negative height information**

We are going to use `Relative_Height_Estimation.py`, which will return a binary raster/array with values of 1 given to pixels at a lower elevation than the elevation associated with NHD MR Flowline pixels and a value of zero to all other pixels, i.e. pixels at a higher elevation than the NHD MR Flowlines. 

This operation will produce three more outputs; the `_NegaHand` is the relative height we are looking for to be added to the cost function, while the other two (`_nhdflowline` and `_Allocation`) are the rasterized NHD flowline used for calculation and another term used for identifying the corridor of values below the elevation the NHD.


In [ ]:
!python geoflood_demo/GeoFlood/GeoFlood/Relative_Height_Estimation.py

---
# **Network extraction with negative height**

We can now run our `Network_Extraction.py` to obtain the geodesic curve between the end points. 


In [ ]:
!python geoflood_demo/GeoFlood/GeoFlood/Network_Extraction.py

---
# **(9) TauDEM preparation**

TauDEM can leverage a computer's resources by running parallel processes.
We can see from the command below the number of cores available. 

In [ ]:
!lscpu | grep 'per socket'

Let's save the number of cores in order to include it in our TauDEM operations

In [ ]:
numcores = !lscpu | grep 'per socket'
numcores = int(numcores[0][-1])
numcores

---
# **(10) TauDEM operations to prepare for the HAND raster calculation**

We will now compute the HAND raster. As it is based on Dinf flow directions, we will need to perform pit filling on the elevation data and compute flow directions. Note we are running with 8 processes.

---
 - **Pit filling**

In [ ]:
!mpiexec -n $numcores pitremove \
    -z geoflood_demo/INPUT/GIS/OnionCreek/OC1mTest.tif \
    -fel geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_fel.tif

---
 - **Dinf flow directions**

In [ ]:
!mpiexec -n $numcores dinfflowdir \
    -fel geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_fel.tif \
    -ang geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_ang.tif \
    -slp geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_slp.tif

---
# **(11) HAND calculation with GeoFlood flowline**

We will now use the previous outputs from TauDEM to compute the HAND raster based on the GeoFlood flowline. 

In [ ]:
!mpiexec -n $numcores dinfdistdown \
    -ang geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_ang.tif \
    -fel geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_fel.tif \
    -slp geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_slp.tif \
    -src geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_path.tif \
    -dd geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_hand_GeoFlood.tif \
    -m ave v

---
# **(12) Calculation of hydraulic geometry for the stream reach**

Now that we have the HAND raster, we will go back to GeoFlood functions to compute the hydraulic geometry properties for this river reach, which will then be used to translate the discharge forecast into a depth prediction for inundation mapping (in the next lab).

The first operation we are going to perform is the segmentation of the flowline which is done within `Streamline_Segmentation.py`. This operation checks the length of the flowline; if the length is longer than a predefined maximum segment length, the script breaks the original flowline into equal-length segments with a length shorter than the predefined threshold. The input of this script is the extracted river network shapefile (`OC1mTest_channelNetwork.shp`), and the output is a river segment shapefile (`OC1mTest_channelSegment.shp`).



---

# **Channel segmentation**

In [ ]:
!python geoflood_demo/GeoFlood/GeoFlood/Streamline_Segmentation.py



---
# **Catchment delineation**

Now we will delineate the catchment for the segmented flowline. Similarly to what we have done in the past, the script uses a GRASS function called from `Segment_Catchment_Delineation.py`.

The river segment shapefile (`OC1mTest_channelSegment.shp`) is one of the script inputs. Since this script relies on the GRASS GIS catchment delineation routine, the flow direction raster (`OC1mTest_fdr.tif`) we derived with GRASS is another input. The output of this script is a segment catchment geotiff file (`OC1mTest_segmentCatchment.tif`).

In [ ]:
!python geoflood_demo/GeoFlood/GeoFlood/Grass_Delineation_py3.py



---


# **Calculation of hydraulic geometry properties**

We can now use the segments, the delineated catchments, and the HAND raster to obtain the hydraulic geometry properties of each of these segments. We will start by running the GeoFlood script `River_Attribute_Estimation.py`, which is used to compute bed slope, river length, and catchment area. The inputs to this script are the DEM in the input folder, the river segments we have previously identified, and the segment catchment raster we just got from GRASS in our `OUTPUT` folder. This script will output a segment catchment polygon shapefile and a river attribute text file, which will be placed in the Hydraulics folder in your `OUTPUT/Hydraulics` folder.

You will find new outputs in your Outputs folder. In particular, in the GIS folder you will find the _segmentCatchment.shp and in the Hydraulics folder you will find the river attribute text file.

In [ ]:
!python geoflood_demo/GeoFlood/GeoFlood/River_Attribute_Estimation.py

---
# **Associate channel roughness to segments**
We will now run the script `Network_Mapping.py` which takes the channel roughness information from the original river network and saves it in the segmented river network. The script takes as input the original catchment shapefile, which is the new input we just added as part of this lab (`Catchment.shp`) and the river segment shapefile we have created (`OC1mTest_channelSegment.shp`).

This script will create a new output in your `OUTPUT` GIS folder which has csv format and contains the COMID of each of the segments in our flowline.

In [ ]:
!python geoflood_demo/GeoFlood/GeoFlood/Network_Mapping.py

---
# **Derivation of channel hydraulic properties**
The next script is part of TauDEM and is called CatchHydroGeo. This script is used to derive the channel hydraulic properties using the terrain analysis products. The inputs are the HAND raster we computed (`OC1mTest_HAND.tif`), the segment catchment raster (`OC1mTest_segmentCatchment.tif`), the Dinf slope raster (`OC1mTest_slp.tif`), the river attribute text file (`OC1mTest_River_Attribute.txt`) and the stage range text file we input.

Once the execution is finished, the resulting channel characteristic table with hydraulic properties directly derived from the HAND raster (`hydroprop-basetable.csv`) will be found in the `OUTPUT/Hydraulics` folder and contains the hydraulic geometry properties for each stage value we input.

In [ ]:
!mpiexec -n $numcores catchhydrogeo \
    -hand geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_hand_GeoFlood.tif \
    -catch geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_segmentCatchment.tif \
    -catchlist geoflood_demo/OUTPUT/Hydraulics/OnionCreek/OC1mTest_River_Attribute.txt \
    -slp geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_slp.tif \
    -h geoflood_demo/INPUT/Hydraulics/OnionCreek/stage.txt \
    -table geoflood_demo/OUTPUT/Hydraulics/OnionCreek/hydroprop-basetable.csv

---
# **Derivation of synthetic rating curves**
We have a final GeoFlood script to run, called `Hydraulic_Property_Postprocess.py`, which combines the channel roughness coefficient and the properties that we have obtained in the above steps to derive reach average geometric properties for the reach and synthetic rating curves. The inputs are the COMID-based Manning’s values (`COMID_Roughness.csv`), the network ID mapping csv table (`OC1mTest_networkMapping.csv`), and the base geometric properties (`hydroprop-basetable.csv`).

You will find as output the full channel properties table with synthetic rating curves in the `OUTPUT` Hydraulics folder, called `hydroprop-fulltable.csv`.

In [ ]:
!python geoflood_demo/GeoFlood/GeoFlood/Hydraulic_Property_Postprocess.py

# **(13) Estimation of flood inundation extent and depth**

And finally, what we were all waiting for! We will integrate all the results above to estimate flood inundation extent and depth based on the hydrological prediction provided as input which comes from the National Water Model. 

https://water.noaa.gov/about/nwm

---
# **Assign NWM input forecasted discharge to channel segments**

This first operation takes the NWM forecast that we provided in input and will assign it to each channel segment based on COMID. The script performs the discharge-stage conversion based on the NWM netCDF output (general format: `nwm.txxz.product_type.channel_rt.fxxx.conus.nc`) from the NWM input folder. The script also uses the network id mapping table, which records how the COMIDs are linked to the ids of the extracted river segments. The script also uses the full channel property table, which contains stage-discharge data pairs for each segment used in stage height interpolation. 

The script produces the depth corresponding to the discharge for each segment in both netCDF and CSV formats. The depths files have the same file name as the NWM inputs and they will be stored in the corresponding OUPUT NWM folder. 

Since our stream does not flood very often we are going to overwrite the discharge forecast so that we can make this system flood. 

<mark>**Modify forecasted discharge in Foreast_Table.py**</mark>

Open `Forecast_Table.py` from the left menu - the script is stored within the GeoFlood folder. Double click on the name of the script and uncomment line 63 by removing the symbol `#`. As you can see we are imposing a discharge of 500 to obtain a flood inundation map (do not worry about any of the symbols that are written on that same line after 500). 


In [ ]:
!python geoflood_demo/GeoFlood/GeoFlood/Forecast_Table.py \
    geoflood_demo/INPUT/NWM/OnionCreek/nwm.t00z.analysis_assim.channel_rt.tm01.conus.nc



---
# **Compute flood inundation map corresponding to the forecasted discharge**

We will now quantify the flood extent and depth by using TauDEM's InunMap. 


In [ ]:
!mpiexec -n $numcores inunmap \
         -hand geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_hand_GeoFlood.tif \
         -catch geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_segmentCatchment.tif \
         -forecast geoflood_demo/OUTPUT/NWM/OnionCreek/nwm.t00z.analysis_assim.channel_rt.tm01.conus.nc \
         -mapfile geoflood_demo/OUTPUT/Inundation/OnionCreek/OC1mTest_NWM_inunmap.tif

# **(14) Plot the inundation map!**

We can now have a look at the results here and check our inundation map. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import rasterio.plot

In [ ]:
# open the HAND raster
with rasterio.open('geoflood_demo/OUTPUT/GIS/OnionCreek/OC1mTest_hand_GeoFlood.tif') as src:
    oc_hand_image = src.read(1) # read band 1 into a numpy array

# open the inundation raster
with rasterio.open('geoflood_demo/OUTPUT/Inundation/OnionCreek/OC1mTest_NWM_inunmap.tif') as src:
    oc_inun_image = src.read(1) # read band 1 into a numpy array

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,5))

hand_im = ax[0].imshow(oc_hand_image, cmap='Blues_r', vmin=0, vmax=80)
fig.colorbar(hand_im, ax=ax[0], shrink=0.4)
ax[0].set_title("GeoFlood HAND")
ax[0].set_xlabel('x (m)')
ax[0].set_ylabel('y (m)')

inun_im = ax[1].imshow(oc_inun_image, cmap='Blues_r', vmin=0, vmax=8)
fig.colorbar(inun_im, ax=ax[1], shrink=0.4)
ax[1].set_title("Inundation map from NWM forecast")
ax[1].set_xlabel('x (m)')
ax[1].set_ylabel('y (m)')

fig.tight_layout()

# save figure 
fig.savefig('demo.png', bbox_inches='tight')
# fig.show()